In [86]:
from glob import glob

import asn1tools

In [87]:
files = glob("../**/*.asn", recursive=True)

def parseFiles(files):
  return asn1tools.parse_files(files)

parsed = parseFiles(files)
for f, p in zip(files, parsed):
  print(f)
  print(f"- {p}")

../ETSI-ITS-ASN1/cam_en302637_2/CAM-PDU-Descriptions.asn
- CAM-PDU-Descriptions
../ETSI-ITS-ASN1/cam_en302637_2/cdd/ITS-Container.asn
- ITS-Container


In [88]:
f = "CAM-PDU-Descriptions"
# f = "ITS-Container"

for k, v in parsed[f].items():
  print(k)
  if isinstance(v, dict):
    for e in v:
      print(f"- {e}")
  else:
    print(f"- {v}")

imports
- ITS-Container
types
- CAM
- CoopAwareness
- CamParameters
- HighFrequencyContainer
- LowFrequencyContainer
- SpecialVehicleContainer
- BasicContainer
- BasicVehicleContainerHighFrequency
- BasicVehicleContainerLowFrequency
- PublicTransportContainer
- SpecialTransportContainer
- DangerousGoodsContainer
- RoadWorksContainerBasic
- RescueContainer
- EmergencyContainer
- SafetyCarContainer
- RSUContainerHighFrequency
- GenerationDeltaTime
values
object-classes
object-sets
extensibility-implied
- False
tags
- AUTOMATIC


In [89]:
types = parsed[f]["types"]

# for k in ["CAM", "CoopAwareness", "CamParameters", "HighFrequencyContainer", "GenerationDeltaTime"]:
#   print(k)
#   print(f"  {types[k]}")

In [90]:
type_types = set([v["type"] for v in types.values()])
print(type_types)

member_keys = set()
for v in types.values():
  if "members" in v:
    for m in v["members"]:
      if m is None:
        continue
      member_keys.update(list(m.keys()))
print(member_keys)

{'INTEGER', 'SEQUENCE', 'CHOICE'}
{'type', 'name', 'optional'}


In [91]:
naive_types = {k: v for k, v in types.items() if v["type"] not in ("SEQUENCE", "SEQUENCE OF", "CHOICE")}
array_types = {k: v for k, v in types.items() if v["type"] == "SEQUENCE OF"}

print(list(naive_types.keys()))
print(list(array_types.keys()))

['GenerationDeltaTime']
[]


In [92]:
def ansType2RosMsg(type_name, type_info):

  if type_name in naive_types or type_name in array_types:
    return

  print(type_name)
  type_type = type_info["type"]
  if type_type in ("SEQUENCE", "CHOICE"):
    if type_type == "CHOICE":
      print()
      print(f"  INTEGER type")
    for im, member in enumerate(type_info["members"]):
      if member is None:
        continue
      if member["type"] in naive_types:
        member_name = member["name"]
        member = naive_types[member["type"]]
        member["name"] = member_name
      if member["type"] in array_types:
        member_name = member["name"]
        member_name = member["name"]
        member = array_types[member['type']]
        member["name"] = member_name
        member["type"] = f"{member['element']['type']}[]"
      lines = []
      comment_lines = []
      lines.append(f"  {member['type']} {member['name']}")
      if type_type == "CHOICE":
        lines.append(f"  INTEGER IS_{member['name'].upper().replace('-', '_')} = {im}")
      if "values" in member:
        for val in member["values"]:
          if val is None:
            continue
          k, v = val
          lines.append(f"  INTEGER {member['name'].upper()}_{k.upper().replace('-', '_')} = {v}")
      if "named-numbers" in member:
        for k, v in member["named-numbers"].items():
          lines.append(f"  {member['type']} {member['name'].upper()}_{k.upper().replace('-', '_')} = {v}")
      for k, v in member.items():
        if k not in ("type", "name", "values", "named-numbers", "element"):
          comment_lines.append(f"{k}: {v}")
      print()
      comment_lines = [f"  # {cl}" for cl in comment_lines]
      for cl in comment_lines:
        print(cl)
      for l in lines:
        print(l)
  else:
    raise ValueError(f"Unhandled type {type_type}")

for k, v in types.items():
  ansType2RosMsg(k, v)
  print("----------------------------------------")

CAM

  ItsPduHeader header

  CoopAwareness cam
----------------------------------------
CoopAwareness

  # restricted-to: [(0, 65535)]
  INTEGER generationDeltaTime
  INTEGER GENERATIONDELTATIME_ONEMILLISEC = 1

  CamParameters camParameters
----------------------------------------
CamParameters

  BasicContainer basicContainer

  HighFrequencyContainer highFrequencyContainer

  # optional: True
  LowFrequencyContainer lowFrequencyContainer

  # optional: True
  SpecialVehicleContainer specialVehicleContainer
----------------------------------------
HighFrequencyContainer

  INTEGER type

  BasicVehicleContainerHighFrequency basicVehicleContainerHighFrequency
  INTEGER IS_BASICVEHICLECONTAINERHIGHFREQUENCY = 0

  RSUContainerHighFrequency rsuContainerHighFrequency
  INTEGER IS_RSUCONTAINERHIGHFREQUENCY = 1
----------------------------------------
LowFrequencyContainer

  INTEGER type

  BasicVehicleContainerLowFrequency basicVehicleContainerLowFrequency
  INTEGER IS_BASICVEHICLECONTAI